In [37]:
from random import seed, shuffle
from collections import defaultdict
from itertools import chain

def number_to_card(n):
    """将牌号转换为Unicode字符"""
    suit_map = defaultdict(
        lambda: 0x1F0B1,  # 默认：红桃
        {
            0: 0x1F0B1,  # 红桃
            1: 0x1F0C1,  # 方块
            2: 0x1F0D1,  # 梅花
            3: 0x1F0A1   # 黑桃
        }
    )
    rank_offset = [0, 6, 7, 8, 9, 10, 12, 13]  # A,7,8,9,10,J,Q,K
    
    suit, rank = n // 8, n % 8  # 计算花色和牌面值
    
    return chr(suit_map[suit] + rank_offset[rank])  # 返回对应的Unicode字符

def distribute_cards(number, all_keeped_cards):
    stacks =[[] for _ in range(number)]
    all_keeped_cards.reverse()
    for index in range(len(all_keeped_cards)):
        stacks[index % number].append(all_keeped_cards[index])
    return stacks

def display_stacks(stacks, show_last = False, stack_num_show = 0): 
    result = ""
    stack_num = 1
    for stack in stacks:
        if len(stack) ==0:
            result += " " * 12
        elif show_last and stack_num_show == stack_num:
            result += '[' * (len(stack) - 1) + number_to_card(stack[-1]) + " " * (12 - len(stack))
        else:
            result += ']' *len(stack) + " " * (12 - len(stack))
        stack_num += 1
    return result
        
def display_discarded_cards(all_discarded_cards):
    if all_discarded_cards:
        return '[' * (len(all_discarded_cards) - 1) + number_to_card(all_discarded_cards[-1])
    return ""
    
def is_ace(card):
    return (card % 8 == 0)

    
def run_one_round(number, all_keeped_cards, all_discarded_cards):
    #分组
    result = []
    if number == 4:
        result.append("Distributing the cards in the deck into 4 stacks.")
    else:
        result.append(f"Distributing the cards that have been kept into {number} stacks.")
       # 分牌 
    stacks = distribute_cards(number, all_keeped_cards)
    
    # 显示初始堆状态
    result.append(display_stacks(stacks))
    result.append("")  # 添加空行以提高可读性
    result.append(display_discarded_cards(all_discarded_cards))
    result.append("")
    result.append("")
    
    number_to_word = dict(zip(range(1,9), ["First", "Second", "Third", "Fourth", "Fifth", "Sixth", "Seventh", "Eighth"]))
    all_keeped_cards = []
    discard_stacks = [[] for _ in range(number)]
    
    for stack_num in range(1, number + 1):
        stack = stacks[stack_num - 1]
        stack.reverse()
        discard_stack = discard_stacks[stack_num - 1]
        
        # 清理当前堆使得要么最后是A要么删除干净
        turn_over = 1
        while stack:
            if is_ace(stack[-1]):
                break
            discard_stack.append(stack.pop())
            turn_over += 1
        if len(stack) == 0:
            result.append(f"No ace in {number_to_word[stack_num].lower()} stack, after it has been turned over.")
        else:
            #这里可能写得不对
            if len(stack) == 1:
                result.append(f"{number_to_word[turn_over]} (and last) card in {number_to_word[stack_num].lower()} stack, after it has been turned over, is an ace.")
            else:
                result.append(f"{number_to_word[turn_over]} card in {number_to_word[stack_num].lower()} stack, after it has been turned over, is an ace.")
        
        result.append(display_stacks(stacks, True, stack_num))
        result.append(display_stacks(discard_stacks, True, stack_num))
        result.append(display_discarded_cards(all_discarded_cards))
        result.append("]" * len(all_keeped_cards))
        result.append("")

# 建议封装函数
        if len(stack)== 0:
            if all_discarded_cards:
                result.append("Adding to the cards that have been discarded all cards in the stack.")
            else:
                result.append("Discarding all cards in the stack.")
        elif len(discard_stack)== 1:
            if all_discarded_cards:
                result.append("Adding to the cards that have been discarded the card before the ace.")
            else:
                result.append("Discarding the card before the ace.")
        elif len(discard_stack)!= 0:
            if all_discarded_cards:
                result.append(f"Adding to the cards that have been discarded the {len(discard_stack)} cards before the ace.")
            else:
                result.append(f"Discarding the {len(discard_stack)} cards before the ace.")

        all_discarded_cards.extend(discard_stack)
        discard_stack.clear()
        
        if len(stack)== 1:
            if not all_keeped_cards:
                result.append("Keeping the ace, turning it over.")
            else:
                result.append("Also keeping the ace, turning it over.")
        elif len(stack) == 2:
            if not all_keeped_cards:
                result.append("Keeping the ace and the card after, turning them over.")
            else:
                result.append("Also keeping the ace and the card after, turning them over.")
        elif len(stack) != 0:
            if not all_keeped_cards:
                result.append(f"Keeping the ace and the {len(stack)- 1} cards after, turning them over.")
            else:
                result.append(f"Also keeping the ace and the {len(stack)- 1} cards after, turning them over.")
        
        all_keeped_cards.extend(stack[::-1])
        stack.clear()
        
        
        result.append(display_stacks(stacks))
        result.append("")
        result.append(display_discarded_cards(all_discarded_cards))
        result.append("]" * len(all_keeped_cards))
        result.append("")
        
    return result, all_keeped_cards, all_discarded_cards

    
def run_one_game(random_seed):
    # Initialization
    all_keeped_cards = list(range(32))
    seed(random_seed)
    shuffle(all_keeped_cards)
    
    result = []
    result.append("")
    result.append("Deck shuffled, ready to start!")
    result.append("]" * 32)  # 显示初始牌堆状态
    result.append("")
    
    
    all_discarded_cards = []
    # 处理三个阶段
    for number in range (4, 1, -1):
        print_result, keeped_cards, discarded_cards = run_one_round(number, all_keeped_cards, all_discarded_cards)
        all_keeped_cards = keeped_cards
        all_discarded_cards = discarded_cards
        result.extend(print_result)

    is_win = False
    result.append(f"Displaying the {len(all_keeped_cards)} cards that have been kept.")
    if set(all_keeped_cards[:4])=={0,8,16,24}:
        result.append("You won!")
        is_win = True
    else:
        result.append("You lost!")
    result.append("")
    result.append("")
    result.append(display_discarded_cards(all_discarded_cards))
    last_line = ""
    for card in all_keeped_cards[::-1]:
        last_line += number_to_card(card)
    result.append(last_line)
    return result,is_win,len(all_keeped_cards)

def simulate(times, i):
    all_results = {}
    for time in range(times):
        result, is_win, keeped_cards_number = run_one_game(i + time)
        if is_win:
            if keeped_cards_number not in all_results:
                all_results[keeped_cards_number] = 1
            else:
                all_results[keeped_cards_number] = all_results[keeped_cards_number] + 1
            # all_results[keeped_cards_number] = all_results.get(keeped_cards_number, 0) + 1
    print("Number of cards left when winning | Frequency")
    print("---------------------------------------------")
    for number in sorted(all_results):
        frequency = all_results[number] / times
        print(f"{number:33} | {frequency:9.2%}")
    
if __name__ == "__main__":
    user_seed = int(input("Please enter an integer to feed the seed() function: "))
    result, is_win, keeped_cards_number = run_one_game(user_seed)
    for line in result:
        print(line.rstrip())

Please enter an integer to feed the seed() function:  0


]]]]]]]]    ]]]]]]]]    ]]]]]]]]    ]]]]]]]]    
[[[[[[[🂡    ]]]]]]]]    ]]]]]]]]    ]]]]]]]]    
                                                
            ]]]]]]]]    ]]]]]]]]    ]]]]]]]]    
            [[[[[[🃁     ]]]]]]]]    ]]]]]]]]    
            🂮                                   
                        ]]]]]]]]    ]]]]]]]]    
                        [[[[[[🃑     ]]]]]]]]    
                        🂪                       
                                    ]]]]]]]]    
                                                
                                    [[[[[[[🂹    
                                                
]]]]]]]]    ]]]]]]]     ]]]]]]]                 
[[[[[🃑      ]]]]]]]     ]]]]]]]                 
[🃞                                              
            ]]]]]]]     ]]]]]]]                 
            [[🃁         ]]]]]]]                 
            [[[🂸                                
                        ]]]]]]]                 
                    

In [ ]:
from random import seed, shuffle

class CardGame:
    @staticmethod
    def number_to_card(card_number):
        # Map suits to Unicode base values (Spades, Hearts, Diamonds, Clubs)
        suit_to_unicode = {
            0: 0x1F0B1,  # Spades
            1: 0x1F0C1,  # Hearts
            2: 0x1F0D1,  # Diamonds
            3: 0x1F0A1   # Clubs
        }
        # Rank offsets for A,7,8,9,10,J,Q,K
        rank_offsets = [0, 6, 7, 8, 9, 10, 12, 13]
        suit, rank = divmod(card_number, 8)
        return chr(suit_to_unicode.get(suit, 0x1F0B1) + rank_offsets[rank])
    
    @staticmethod
    def distribute_cards(num_stacks, card_deck):
        stacks = [[] for _ in range(num_stacks)]
        for i, card in enumerate(reversed(card_deck)):
            stacks[i % num_stacks].append(card)
        return stacks

    @classmethod
    def display_stacks(cls, card_stacks, show_last=False, stack_to_show=0):
        display = ""
        for i, stack in enumerate(card_stacks, start=1):
            if not stack:
                display += " " * 12
            elif show_last and stack_to_show == i:
                display += '[' * (len(stack)-1) + cls.number_to_card(stack[-1]) + " "*(12-len(stack))
            else:
                display += ']' * len(stack) + " "*(12-len(stack))
        return display
    
    @classmethod
    def display_discarded(cls, discarded_cards):
        return ('['*(len(discarded_cards)-1) + cls.number_to_card(discarded_cards[-1])) if discarded_cards else ""

    @staticmethod
    def is_ace(card_number):
        return card_number % 8 == 0

    @classmethod
    def play_round(cls, num_stacks, kept_cards, discarded_cards):
        output = []
        output.append("Distributing the cards " + 
                     ("in the deck into 4 stacks." if num_stacks==4 else 
                      f"that have been kept into {num_stacks} stacks."))
        
        stacks = cls.distribute_cards(num_stacks, kept_cards)
        output.append(cls.display_stacks(stacks))
        output.append("")
        output.append(cls.display_discarded(discarded_cards))
        output.append("")
        output.append("")
        
        # Map numbers to ordinal words (1st, 2nd, etc.)
        ordinal_words = {
            1: "First", 2: "Second", 3: "Third", 4: "Fourth",
            5: "Fifth", 6: "Sixth", 7: "Seventh", 8: "Eighth"
        }
        
        new_kept_cards = []
        temporary_discards = [[] for _ in range(num_stacks)]
        
        for stack_idx in range(1, num_stacks+1):
            current_stack = stacks[stack_idx-1]
            current_stack.reverse()
            current_temp_discard = temporary_discards[stack_idx-1]
            
            turn_count = 1
            while current_stack:
                if cls.is_ace(current_stack[-1]):
                    break
                current_temp_discard.append(current_stack.pop())
                turn_count += 1
                
            if not current_stack:
                output.append(f"No ace in {ordinal_words[stack_idx].lower()} stack, after it has been turned over.")
            else:
                if len(current_stack) == 1:
                    output.append(f"{ordinal_words[turn_count]} (and last) card in {ordinal_words[stack_idx].lower()} stack, after it has been turned over, is an ace.")
                else:
                    output.append(f"{ordinal_words[turn_count]} card in {ordinal_words[stack_idx].lower()} stack, after it has been turned over, is an ace.")
                    
            output.append(cls.display_stacks(stacks, True, stack_idx))
            output.append(cls.display_stacks(temporary_discards, True, stack_idx))
            output.append(cls.display_discarded(discarded_cards))
            output.append("]" * len(new_kept_cards))
            output.append("")
            
            if len(current_stack) == 0:
                output.append("Adding to the cards that have been discarded all cards in the stack." if discarded_cards else "Discarding all cards in the stack.")
            elif len(current_temp_discard) == 1:
                output.append("Adding to the cards that have been discarded the card before the ace." if discarded_cards else "Discarding the card before the ace.")
            elif current_temp_discard:
                output.append((f"Adding to the cards that have been discarded the {len(current_temp_discard)} cards before the ace."
                             if discarded_cards else f"Discarding the {len(current_temp_discard)} cards before the ace."))
                
            discarded_cards.extend(current_temp_discard)
            current_temp_discard.clear()
            
            if len(current_stack) == 1:
                output.append("Keeping the ace, turning it over." if not new_kept_cards else "Also keeping the ace, turning it over.")
            elif len(current_stack) == 2:
                output.append("Keeping the ace and the card after, turning them over." if not new_kept_cards else "Also keeping the ace and the card after, turning them over.")
            elif current_stack:
                output.append((f"Keeping the ace and the {len(current_stack)-1} cards after, turning them over."
                             if not new_kept_cards else f"Also keeping the ace and the {len(current_stack)-1} cards after, turning them over."))
                
            new_kept_cards.extend(current_stack[::-1])
            current_stack.clear()
            
            output.append(cls.display_stacks(stacks))
            output.append("")
            output.append(cls.display_discarded(discarded_cards))
            output.append("]" * len(new_kept_cards))
            output.append("")
            
        return output, new_kept_cards, discarded_cards

    @classmethod
    def play_game(cls, seed_value):
        kept_cards = list(range(32))
        seed(seed_value)
        shuffle(kept_cards)
        
        game_output = ["", "Deck shuffled, ready to start!", "]"*32, ""]
        discarded_cards = []
        
        for num_stacks in range(4, 1, -1):
            round_output, kept_cards, discarded_cards = cls.play_round(num_stacks, kept_cards, discarded_cards)
            game_output.extend(round_output)
            
        game_output.append(f"Displaying the {len(kept_cards)} cards that have been kept.")
        win = (set(kept_cards[:4]) == {0, 8, 16, 24})  # Check if all four aces are on top
        game_output.append("You won!" if win else "You lost!")
        game_output.append("")
        game_output.append("")
        game_output.append(cls.display_discarded(discarded_cards))
        
        final_cards = "".join(cls.number_to_card(card) for card in reversed(kept_cards))
        game_output.append(final_cards)
        
        return game_output, win, len(kept_cards)
    
    @classmethod
    def run_simulations(cls, num_simulations, starting_seed):
        results = {}
        for i in range(num_simulations):
            _, win, cards_left = cls.play_game(starting_seed + i)
            if win:
                results[cards_left] = results.get(cards_left, 0) + 1
                
        print("Number of cards left when winning | Frequency")
        print("---------------------------------------------")
        for cards_left in sorted(results):
            print(f"{cards_left:33} | {results[cards_left]/num_simulations:9.2%}")

# Alias for convenience
simulate = CardGame.run_simulations

if __name__ == "__main__":
    seed_value = int(input("Please enter an integer to feed the seed() function: "))
    game_output, win_status, cards_left = CardGame.play_game(seed_value)
    for line in game_output:
        print(line.rstrip())